# AlphaZero Algorithm - Testing of the buffer

# Import libraries / modules

In [1]:
# Python libraries
import time

# 3rd party libraries
import matplotlib.pyplot as plt
import numpy as np

# Game-related libraries
import games_mod # Games
import policy_mod # neural network
from play_mod import Play
import training_mod
from game_utils import DotDict, policy_player_mcts, random_player, match_ai

# Game, Training and Play Settings

In [2]:
# Game settings
game_settings = DotDict({
    "board_size": (3,3),
    "N": 3
})

# Self-play training settings
game_training_settings = DotDict({
    "comp_interval":100,
    "episods": 1000,
    "explore_steps": 50,
    "temp_threshold": [300, 0.01] #[x,y] means "up to x episods, applies y temperature"
})
# neural network settings
nn_training_settings = DotDict({
    "load_policy": False,
    "ai_ckp": "",
    "lr": .01, 
    "weight_decay": 1.e-4,
})

# play settings
play_settings = DotDict({
    "explore_steps": 50,
    "temperature": 0.01                         
})

## Testing of the buffer

In [3]:
from replay_buffer import ReplayBuffer
buffer_size = 100
batch_size = 4
buffer = ReplayBuffer(buffer_size, batch_size)

In [4]:
game=games_mod.ConnectN(game_settings)

In [18]:
game.state

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [19]:
game.move((1,1))

True

In [20]:
game.state

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [21]:
from self_play import execute_self_play
policy = policy_mod.Policy()
temp = 1
wins = 0
draws = 0
losses = 0
explore_steps = 10
new_exp = execute_self_play (game_settings, explore_steps, policy, temp)

In [22]:
for i in range(len(new_exp)):
    buffer.add(new_exp[i])

In [26]:
buffer.buffer_len()

5

In [27]:
samples = buffer.sample()

In [28]:
samples

[Experience(state=array([[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.]]), v=1, p=tensor([ 0.0000,  0.1818,  0.1818,  0.1818,  0.0909,  0.0000,  0.0000,
          0.1818,  0.1818])),
 Experience(state=array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]), v=1, p=tensor([ 0.2222,  0.0000,  0.0000,  0.1111,  0.0000,  0.2222,  0.2222,
          0.2222,  0.0000])),
 Experience(state=array([[ 0., -1.,  1.],
        [ 0.,  0.,  1.],
        [ 0., -1.,  0.]]), v=1, p=tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])),
 Experience(state=array([[ 0., -1.,  1.],
        [ 0.,  0.,  1.],
        [ 0.,  0.,  0.]]), v=1, p=tensor([ 0.2000,  0.0000,  0.0000,  0.2000,  0.2000,  0.0000,  0.0000,
          0.2000,  0.2000]))]

In [13]:
import torch
import torch.nn as nn

In [33]:
policy = policy_mod.Policy()

In [34]:
from mcts import process_policy

In [1]:
# test with 1 state

input1 = samples[0][0]
target1 = samples[0][2]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
frame = torch.tensor(input1, dtype=torch.float, device=device)
input = frame.unsqueeze(0).unsqueeze(0)
predicted = policy(input)[0]

NameError: name 'samples' is not defined

In [49]:
target = torch.tensor(target1, dtype=torch.float, device=device).unsqueeze(0).unsqueeze(0)
mse_loss = nn.MSELoss()
predicted = predicted.view(-1, 9)
output = mse_loss(predicted, target)
#output.backward()

RuntimeError: input and target shapes do not match: input [1 x 9], target [1 x 1 x 8] at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/THNN/generic/MSECriterion.c:13